In [1]:
import os

repo_dir = os.environ.get("REPO_DIR")
code_dir = os.path.join(repo_dir, "code/")
data_dir = os.path.join(repo_dir, "data/")

os.chdir(code_dir)

import geopandas as gpd
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import os
import shutil


import sys


from mosaiks.label_utils.utils import geopandas_shape_grid, box_grid, assign_grid_points_to_gpdFile, get_dense_grid_for_gpdf_file
from mosaiks.label_utils.plotting_utils import plot_label_map_hist

# HDI data (Smits et al)

**Data Download:**

*Shapefiles and tabular data are separate downloads*

*Files downloaded on July 17, 2023:*

Tabular data:
https://globaldatalab.org/mygdl/downloads/


https://globaldatalab.org/asset/394/SHDI-SGDI-Total%207.0.csv

We are using the SHDI V7.0data in this analysis. The full database is downloaded from the link above. Version history is [here](https://globaldatalab.org/shdi/archive/).


*A previous version of this manuscript used the V4 version of these labels.*


Shapefiles:
https://globaldatalab.org/shdi/shapefiles/

https://globaldatalab.org/asset/403/GDL%20Shapefiles%20V6.1.zip

We use the neweset shapefile available on July 17, 2023. This is the `GDL Shapefiles V6.1`. This file is NOT included in the GitHub repository and must be downloaded to replicate our data cleaning.



**Data Citation**

Smits, J., Permanyer, I. The Subnational Human Development Database. Sci Data 6, 190038 (2019). https://doi.org/10.1038/sdata.2019.38

**Corresponding paper:**

https://www.nature.com/articles/sdata201938


**Abstract**

In this paper we describe the Subnational Human Development Database. This database contains for the period 1990–2017 for 1625 regions within 161 countries the national and subnational values of the Subnational Human Development Index (SHDI), for the three dimension indices on the basis of which the SHDI is constructed – education, health and standard of living --, and for the four indicators needed to create the dimension indices -- expected years of schooling, mean years of schooling, life expectancy and gross national income per capita. The subnational values of the four indicators were computed using data from statistical offices and from the Area Database of the Global Data Lab, which contains indicators aggregated from household surveys and census datasets. Values for missing years were estimated by interpolation and extrapolation from real data. By normalizing the population-weighted averages of the indicators to their national levels in the UNDP-HDI database, values of the SHDI and its dimension indices were obtained that at national level equal their official versions of the UNDP.


**Data sources**

Three major data sources were used to create our SHDI database. We approached statistical offices, including Eurostat, the statistical office of the European Union (https://ec.europa.eu/eurostat), by email communication or visiting their websites to obtain data. We downloaded data from the Area Database of the Global Data Lab (https://www.globaldatalab.org). And we downloaded data from the HDI website of the Human Development Report Office of the United Nations Development Program (http://hdr.undp.org). In the ‘SHDI Start’ data file (Data Citation 1), for each country information is provided on the data source(s) used for the subnational values of the indicators. In this file also for each country the years for which data is available, the number of subnational regions and the population size is presented. Below we discuss the three main data sources in more detail.



## Read in shape files

In [2]:
directory = data_dir + "raw/GDL_HDI/"
out_directory = data_dir + "int/GDL_HDI/"

## This file MUST be downloaded from the link above and placed in the correct subdirectory


shp_path = directory+"GDL_Shapefiles_V6.1/shdi2022_World_large.shp"

if os.path.exists(shp_path):
    print("reading shp")
    gpdf = gpd.read_file(shp_path)
    
elif os.path.exists(directory+"GDL Shapefiles V6.1.zip"):
    print("unzipping file")
    try:
        os.mkdir(directory+"/GDL_Shapefiles_V6.1/")
    except:
        shutil.unpack_archive(directory+"GDL Shapefiles V6.1.zip", directory+"/GDL_Shapefiles_V6.1")
else:
    print("Shapefile needs to be downnloaded and placed in the correct directory. See details above.")

reading shp


In [3]:
gpdf.rename(columns = {"gdlcode":"GDLcode"}, inplace=True) #Revert to an older name convention

In [4]:
gpdf[gpdf["GDLcode"].isnull()]  # No null GDLcodes

,GDLcode,continent,iso_code,geometry


In [5]:
gpdf.set_index("GDLcode", inplace=True)
gpdf.loc["BHRt","iso_code"] = "BHR" # Fix weird anomaly in shapefile
gpdf.loc[gpdf.index.str.startswith("CUB"),"iso_code"] = "CUB" # fix missing iso code or Cuba
gpdf["iso_code"] = gpdf["iso_code"].replace("XKO","KSV")

In [6]:
nulls = gpdf[gpdf["iso_code"].isnull()] # Make a df of remaining null values in the country code

gpdf.dropna(subset = ["iso_code"],inplace=True)

### Let's make and save a country aggregated version of this shapefiile -- it will be useful later

In [7]:
# gpdf_country = gpdf.dissolve("iso_code")
# gpdf_country.to_pickle(out_directory + "/HDI_ADM0_dissolved_shapefile.p")

In [8]:
gpdf_country = pd.read_pickle(out_directory + "/HDI_ADM0_dissolved_shapefile.p")

## Read and clean data files

See above for details on this tabular data download.

In [51]:
data = pd.read_csv(directory + "/SHDI-SGDI-Total 7.0.csv",low_memory = False)

#Subset to only 2019 observations. This is the year for which we have MOSAIKS features
data = data[data["year"] == 2019]

In [40]:
data[data["GDLCODE"].isin(nulls.index)] # None of the remaining null iso codes have matching HDI values

,iso_code,country,year,GDLCODE,level,region,continent,sgdi,shdi,shdif,...,lifexp,lifexpf,lifexpm,gnic,gnicf,gnicm,lgnic,lgnicf,lgnicm,pop


In [41]:
rename_dictionary = {"shdi" : "Sub-national HDI",
                    "msch": "Mean years schooling",
                    "esch":"Expected years schooling",
                    "lifexp":"Life expectancy",
                    "gnic": "GNI per capita in thousands of US$ (2011 PPP)",
                    "iso_code": "ISO_Code"}

tasks = list(rename_dictionary.values())[:-1]

data.rename(columns = rename_dictionary, inplace=True)

In [44]:
unneeded_cols = ['sgdi', 'shdif', 'shdim',
       'healthindex', 'healthindexf', 'healthindexm', 'incindex', 'incindexf',
       'incindexm', 'edindex', 'edindexf', 'edindexm', 'eschf',
       'eschm', 'mschf', 'mschm', 'gnicf',
       'gnicm', "lgnic", "lgnicf", "lgnicm", "lifexpf", "lifexpm"]

data.drop(columns = unneeded_cols, inplace=True)

data["ISO_Code"] = data["ISO_Code"].replace("XKO","KSV") # Set ISO code for Kosovo. For our use, first 3 of GDLcode

In [45]:
for task in tasks:
    data[task] = pd.to_numeric(data[task], errors="coerce")

In [46]:
national_data_only_indices = data.groupby("ISO_Code").size()

In [47]:
## Now we want to take the countries where we only have national data and merge those with the dataframe of subnational entities
national_data_only_indices = data.groupby("ISO_Code").size()==1
national_data_only = data.groupby("ISO_Code").first()[national_data_only_indices].reset_index()

subnational_data_only = data[data["level"] == "Subnat"]

df = pd.concat([national_data_only, subnational_data_only])

### Let's inspect the set of countries that do not have subnational province observations

In [17]:
#pd.set_option('display.max_rows', None)
print("Countries that do not have ADM1 child regions:")
national_data_only

Countries that do not have ADM1 child regions:


,ISO_Code,country,year,GDLCODE,level,region,continent,Sub-national HDI,Expected years schooling,Mean years schooling,Life expectancy,GNI per capita in thousands of US$ (2011 PPP),pop
0,AND,Andorra,2019,ANDt,National,Total,Europe,0.873,13.300,10.555,83.004,54465.047,
1,ARE,United Arab Emirates,2019,AREt,National,Total,Asia/Pacific,0.920,15.694,12.694,79.726,68590.901,
2,ATG,Antigua and Barbuda,2019,ATGt,National,Total,America,0.800,14.185,9.294,78.691,20600.959,
3,BHR,Bahrain,2019,BHRt,National,Total,Asia/Pacific,0.882,16.298,10.797,80.019,42664.292,
4,BHS,Bahamas,2019,BHSt,National,Total,America,0.816,12.899,12.642,71.205,34676.990,
5,BRN,Brunei Darussalam,2019,BRNt,National,Total,Asia/Pacific,0.830,14.064,9.180,74.748,63963.475,
6,CYP,Cyprus,2019,CYPt,National,Total,Europe,0.897,15.648,12.362,81.397,39568.032,
7,DMA,Dominica,2019,DMAt,National,Total,America,0.729,13.319,8.142,73.559,12694.121,
8,FSM,Micronesia (Federated States of),2019,FSMt,National,Total,Asia/Pacific,0.633,11.549,7.805,71.077,3937.054,
9,GRD,Grenada,2019,GRDt,National,Total,America,0.800,18.000,9.032,74.863,14983.786,


These are all very small countries and this appears to be reasonable.

### The shapefile is not a perfect match the tabular data

Let's analyze what is missing


#### First, let's inspect the set of countries that cannot be linked to a shapefile primary key

In [18]:
nats_dropped = national_data_only[~national_data_only["GDLCODE"].isin(gpdf.index)]
nats_dropped

,ISO_Code,country,year,GDLCODE,level,region,continent,Sub-national HDI,Expected years schooling,Mean years schooling,Life expectancy,GNI per capita in thousands of US$ (2011 PPP),pop
12,KNA,Saint Kitts and Nevis,2019,KNAt,National,Total,America,0.783,15.428,8.663,71.572,26727.539,


These is a vry small country. Excluding this from our analysis seems reasonable.

#### Second, let's inspect the set of ADM1 polygons that cannot be linked to a shapefile primary key

In [19]:
subnats_dropped = subnational_data_only[~subnational_data_only.GDLCODE.isin(gpdf.index)]
subnats_dropped

,ISO_Code,country,year,GDLCODE,level,region,continent,Sub-national HDI,Expected years schooling,Mean years schooling,Life expectancy,GNI per capita in thousands of US$ (2011 PPP),pop
16648,ESP,Spain,2019,ESPr117,Subnat,Ciudad Autonoma de Ceuta,Europe,0.860,16.890,9.256,80.967,31937.756,
16649,ESP,Spain,2019,ESPr118,Subnat,Ciudad Autonoma de Melilla,Europe,0.855,16.279,9.778,80.967,29234.032,
17659,FJI,Fiji,2019,FJIr101,Subnat,Naitasiri,Asia/Pacific,0.763,15.592,11.294,67.523,14258.554,
17660,FJI,Fiji,2019,FJIr102,Subnat,Rewa,Asia/Pacific,0.754,15.250,11.506,66.789,13270.149,
17661,FJI,Fiji,2019,FJIr103,Subnat,"Serua, Namosi",Asia/Pacific,0.722,13.771,11.143,65.610,11094.674,
17662,FJI,Fiji,2019,FJIr104,Subnat,Tailevu,Asia/Pacific,0.738,14.373,11.378,67.166,11046.140,
17663,FJI,Fiji,2019,FJIr105,Subnat,"Kadavu, Lau, Lomaiviti, Rotuma",Asia/Pacific,0.696,13.143,10.788,65.829,7864.513,
17664,FJI,Fiji,2019,FJIr106,Subnat,"Cakaudrove, Bua",Asia/Pacific,0.701,13.041,10.188,67.279,8670.336,
17665,FJI,Fiji,2019,FJIr107,Subnat,Macuata,Asia/Pacific,0.735,15.348,10.226,65.536,13212.098,
17666,FJI,Fiji,2019,FJIr108,Subnat,Ba,Asia/Pacific,0.751,14.396,10.890,67.926,14686.812,


In [20]:
len(subnats_dropped)

47

Dropping these 47 subnational observations is the best we can do. Some appear quite reasonable (e.g., it probably doesn't make sense to consider Guadeloupe a part of France for the purpose of this analysis).

### Now let's see if there is any data in the shapefile that is missing from the tabular data

In [21]:
print("Shape file obs that don't match tabular data")

gpdf[~gpdf.index.isin(df.GDLCODE)] # Just a few

Shape file obs that don't match tabular data


,continent,iso_code,geometry
GDLcode,,,
CHNr132,Asia/Pacific,CHN,"MULTIPOLYGON (((114.24903 22.15958, 114.24903 ..."
GEOr112,Europe,GEO,"POLYGON ((44.68073 41.70555, 44.68641 41.71712..."
TONt,Asia/Pacific,TON,"MULTIPOLYGON (((-176.20329 -22.33513, -176.203..."
WSMt,Asia/Pacific,WSM,"MULTIPOLYGON (((-171.40791 -14.07395, -171.407..."


In [22]:
n_dropped = len(nats_dropped) + len(subnats_dropped)

## Let's go ahead and subset both of these files to the matching set of indices

In [23]:
df.set_index("GDLCODE", inplace=True)
#gpdf.set_index("GDLcode", inplace=True)

In [24]:
matching_locs = df.index[df.index.isin(gpdf.index)]

In [25]:
df = df.loc[matching_locs]
gpdf = gpdf.loc[matching_locs]

In [28]:
df

,ISO_Code,country,year,level,region,continent,Sub-national HDI,Expected years schooling,Mean years schooling,Life expectancy,GNI per capita in thousands of US$ (2011 PPP),pop
GDLCODE,,,,,,,,,,,,
ANDt,AND,Andorra,2019,National,Total,Europe,0.873,13.300,10.555,83.004,54465.047,
AREt,ARE,United Arab Emirates,2019,National,Total,Asia/Pacific,0.920,15.694,12.694,79.726,68590.901,
ATGt,ATG,Antigua and Barbuda,2019,National,Total,America,0.800,14.185,9.294,78.691,20600.959,
BHRt,BHR,Bahrain,2019,National,Total,Asia/Pacific,0.882,16.298,10.797,80.019,42664.292,
BHSt,BHS,Bahamas,2019,National,Total,America,0.816,12.899,12.642,71.205,34676.990,
...,...,...,...,...,...,...,...,...,...,...,...,...
ZWEr106,ZWE,Zimbabwe,2019,Subnat,Matebeleland South,Africa,0.592,10.989,7.834,65.423,3222.317,
ZWEr107,ZWE,Zimbabwe,2019,Subnat,Midlands,Africa,0.598,12.668,8.492,60.559,3540.908,
ZWEr108,ZWE,Zimbabwe,2019,Subnat,Masvingo,Africa,0.596,12.058,8.013,64.420,3019.013,


In [ ]:
df.to_pickle(out_directory + "/HDI_indicators_and_indices_clean.p")
gpdf.to_pickle(out_directory + "/HDI_ADM1_shapefile_clean.p")

#### Also write the national level data

In [33]:
nat_data = data[data["level"] == "National"].set_index("ISO_Code")
nat_data.loc[gpdf_country.index] # Only include countries that also have a shapefile
nat_data.to_pickle(out_directory + "/HDI_indicators_and_indices_adm0_clean.p")

In [34]:
len(matching_locs)

1759

In [35]:
print(round(n_dropped/(len(matching_locs ) + n_dropped),3) * 100, "% of HDI data dropped")

2.7 % of HDI data dropped


##  Transform shapefile to .01 x . 01 degree grid

This is the form needed for aggregating features in the existing feature aggregation pipeline

In [ ]:
gpdf.head()

In [ ]:
# dense_grid = get_dense_grid_for_gpdf_file(gpdf.reset_index(), columns=["GDLCODE", "iso_code"])

In [ ]:
# dense_grid.head()

In [ ]:
# outpath = data_dir + "/features/prepared_labels/GDL_HDI_polygon_coords_for_featurization.p"
# dense_grid["constant"] = 1


# dense_grid.to_pickle(outpath)
# dense_grid = pd.read_pickle(outpath)

Check to see if any polygon observations were dropped. This would occur if they are very small and don't overlay any grid centorids.

In [1]:
# len(dense_grid["GDLCODE"].unique()) == len(matching_locs)